# Music  Antiquing, Part 1 — ECSE 351, Spring 2025

This is the first part of the Music Antiquing exercise. Over the course of the semester, we'll use Python to add effects to a sound clip to make it sound like it's playing on a gramophone. Here's where the goalposts are: https://youtu.be/QbsXLDNPvNc?t=28 

Use the code below and a favorite piece of music that has good dynamic range and frequency span.  Make a one-minute monaural sound file from it (e.g., a .wav file).   Filter the music to a gramophone channel (160 Hz to 2000 Hz), to a telephone channel (300-3000 Hz), to an AM broadcast radio channel (200-5000 Hz), and to an FM broadcast radio channel (30-15,000 Hz). 
Hint: All you need to do is copy the code and change F1 and F2. 

## 📡 Assignment requirements
 - [X] Open the Jupyter notebook.
 - [ ] Upload a .wav file of your choice.
 - [ ] Run the sample code for your .wav file.
 - [ ] Listen to the recordings. Repeat this process for AM and FM broadcast bands by changing the values for F1 and F2. 
 - [ ] Change the code below to filter the .WAV file to a gramophone frequency range. (Also, you'll want to change the code that generates the filename to read "gramophone" instead of "telephone."
 - [ ] Download and save a local copy of your gramophone recording. You'll need it for the next exercise.
 - [ ] Upload your "gramophone" recording here: https://docs.google.com/forms/d/1ICqKKwbJoknCo_0zhtQuk7iVmR31PHFxYuB6FlPYazU/edit
 - [ ] Optional: Save a local copy of the Jupyter notebook itself. (In Binder, your changes and created files will not be saved.)
 - [ ] Under "File" hit "Save and Export Notebook As" and save your file as a PDF.
 - [ ] Print the PDF and staple it to HW1.
 - [ ] On the printed copy, highlight the changes you made to the code.
 - [ ] Annotate the plots. How did the signal change? Does the bandwidth matter? If the signal is going to be bandlimited, do you prefer extending the bass or the treble? Why?


First, we import necessary Python packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import scipy.signal as signal
from scipy.signal import butter, lfilter, freqz
from IPython.display import Audio
import requests

## Play and Plot Original .WAV File
### &#x1F4E1; Input your filename and ID here.

In [ ]:
path = "example.wav"
ID = "KD8OXT" # Put your Case ID here.
name = "YOURNAMEHERE"

# file = input("What is your filename?")
# ID = input("What is your Case ID?")
# name = input ("Type your name here.")

### Option 1: Load a local .WAV file.

(Note: For the example here it's necessary to transpose the data with the .T operator in Line 6 of the following cell. That may or may not be necessary for your WAV file - if you get an error one way, try the other.) 

In [ ]:
# Load audio file
fs, data = wavfile.read(path)
data = data[:,1] # Switch from stereo to mono
# Create and display the Audio object
audio = Audio(data.T, rate=fs) #.
display(audio)

### Option 2: Load a .WAV file from a URL. 
Here's Kristina's test signal on WWV. Click this link for details. [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.5602094.svg)](https://doi.org/10.5281/zenodo.5602094)

The code in the cell below is commented out. To run it: 
 - Put your cursor in the cell
 - Hit CTRL + A to select all the code in the cell
 - Hit CTRL + / to uncomment the code
 - Hit CTRL + ENTER to run the code in the cell

You can adapt this code for a URL of your own, if you wish.

In [ ]:
# url = "https://zenodo.org/records/5602094/files/test.wav?download=1"
# response = requests.get(url)
# # Check if the request was successful
# if response.status_code == 200:
#     # Write the content to a temporary file
#     with open("temp.wav", "wb") as f:
#         f.write(response.content)

#     # Read the WAV file using scipy
#     path = "temp.wav"
#     fs, data = wavfile.read(path)


# Audio(data, rate = fs)

Let's plot the data. 

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(data)
plt.title("Time Domain Plot by  " + name)
plt.subplot(1, 2, 2)
plt.title("Spectrogram by  " + name)
plt.specgram(data)
plt.show()

## Apply filter

In [ ]:
# Define filter parameters

# Telephone
F1 = 300
F2 = 3000

# # AM Broadcast
# F1 = 200
# F2 = 5000

# # FM Broadcast
# F1 = 30
# F2 = 15000

# fs = 44100  # Sample rate (adjust if needed)
Fn = fs / 2  # Nyquist frequency
Wp = [F1/Fn, F2/Fn] 

# Design Butterworth filter
b, a = signal.butter(5, Wp, btype='bandpass')

# Apply filter
filtered_data = signal.lfilter(b, a, data)


In [ ]:
# Calculate FFT
bfilt = np.fft.fft(data)               # Take FFT of original data
afilt = np.fft.fft(filtered_data)      # Take FFT of filtered data

# Plot frequency response
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(np.abs(bfilt[:len(bfilt)//2]))
plt.title('Frequency Response of Unfiltered Signal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

plt.subplot(2, 1, 2)
plt.plot(np.abs(afilt[:len(afilt)//2]))
plt.title('Frequency Response of Filtered Signal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.tight_layout()

In [ ]:
# Reconstruct time-domain signal
foo = np.fft.ifft(afilt)
# Play sound
from IPython.display import Audio
Audio(foo, rate = fs)

In [ ]:
plt.plot(data - data.mean()) # Discard DC offset
plt.plot(foo)
plt.title("Original and Filtered Data from " + name + ": " + str(F1) + " to " + str(F2) + " Hz Bandpass")
plt.show()

In [ ]:
wavfile.write("telophone_" + ID + ".wav", fs, foo.real.astype(np.int16))